<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/Quantum_computing4/blob/main/Final_Estimator_qnn_AnaAlonso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
!pip install qiskit
!pip install qiskit-machine-learning
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.9/306.9 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512575 sha256=0a798a9724487c5b634153bbd4440b9865073007f5cec562

In [22]:
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer

In [23]:
params1 = [Parameter("input1"), Parameter("weight1")]
qc1 = QuantumCircuit(1)
qc1.h(0)
qc1.ry(params1[0], 0)
qc1.rx(params1[1], 0)
circuit_drawer(qc1)

┌───┐┌────────────┐┌─────────────┐
q: ┤ H ├┤ Ry(input1) ├┤ Rx(weight1) ├
   └───┘└────────────┘└─────────────┘

Creating the Estimator qnn with multiple (2 in this case) observables. This will make the QNN architecture more complex.

In [24]:
from qiskit.quantum_info import SparsePauliOp

observable1 = SparsePauliOp.from_list([("Y" * qc1.num_qubits, 1)])
observable2 = SparsePauliOp.from_list([("Z" * qc1.num_qubits, 1)])

estimator_qnn2 = EstimatorQNN(
    circuit=qc1,
    observables=[observable1, observable2],
    input_params=[params1[0]],
    weight_params=[params1[1]],
)

In [25]:
from qiskit_machine_learning.neural_networks import EstimatorQNN

estimator_qnn = EstimatorQNN(
    circuit=qc1,
    observables=[observable1, observable2],
    input_params=[params1[0]],
    weight_params=[params1[1]],
)
estimator_qnn

In [26]:
from qiskit.circuit import ParameterVector

inputs2 = ParameterVector("input", 2)
weights2 = ParameterVector("weight", 4)
print(f"input parameters: {[str(item) for item in inputs2.params]}")
print(f"weight parameters: {[str(item) for item in weights2.params]}")

qc2 = QuantumCircuit(2)
qc2.ry(inputs2[0], 0)
qc2.ry(inputs2[1], 1)
qc2.cx(0, 1)
qc2.ry(weights2[0], 0)
qc2.ry(weights2[1], 1)
qc2.cx(0, 1)
qc2.ry(weights2[2], 0)
qc2.ry(weights2[3], 1)

circuit_drawer(qc2)

input parameters: ['input[0]', 'input[1]']
weight parameters: ['weight[0]', 'weight[1]', 'weight[2]', 'weight[3]']


┌──────────────┐     ┌───────────────┐     ┌───────────────┐
q_0: ┤ Ry(input[0]) ├──■──┤ Ry(weight[0]) ├──■──┤ Ry(weight[2]) ├
     ├──────────────┤┌─┴─┐├───────────────┤┌─┴─┐├───────────────┤
q_1: ┤ Ry(input[1]) ├┤ X ├┤ Ry(weight[1]) ├┤ X ├┤ Ry(weight[3]) ├
     └──────────────┘└───┘└───────────────┘└───┘└───────────────┘

In [27]:
from qiskit_machine_learning.neural_networks import SamplerQNN

sampler_qnn = SamplerQNN(circuit=qc2, input_params=inputs2, weight_params=weights2)
sampler_qnn

In [28]:
from qiskit_algorithms.utils import algorithm_globals

estimator_qnn_input = algorithm_globals.random.random(estimator_qnn.num_inputs)
estimator_qnn_weights = algorithm_globals.random.random(estimator_qnn.num_weights)

print(f"Number of input features for EstimatorQNN: {estimator_qnn.num_inputs} \nInput: {estimator_qnn_input}")
print(f"Number of trainable weights for EstimatorQNN: {estimator_qnn.num_weights} \nWeights: {estimator_qnn_weights}")

Number of input features for EstimatorQNN: 1 
Input: [0.89247533]
Number of trainable weights for EstimatorQNN: 1 
Weights: [0.23502147]


In [29]:
sampler_qnn_input = algorithm_globals.random.random(sampler_qnn.num_inputs)
sampler_qnn_weights = algorithm_globals.random.random(sampler_qnn.num_weights)

print(f"Number of input features for SamplerQNN: {sampler_qnn.num_inputs} \nInput: {sampler_qnn_input}")
print(f"Number of trainable weights for SamplerQNN: {sampler_qnn.num_weights} \nWeights: {sampler_qnn_weights}")

Number of input features for SamplerQNN: 2 
Input: [0.44417614 0.8637556 ]
Number of trainable weights for SamplerQNN: 4 
Weights: [0.31562327 0.49382991 0.2119344  0.82912966]


Batched Forward Pass

In [30]:
estimator_qnn_forward_batched = estimator_qnn.forward(
    [estimator_qnn_input, estimator_qnn_input], estimator_qnn_weights
)

print(f"Forward pass result for EstimatorQNN: {estimator_qnn_forward_batched}.  \nShape: {estimator_qnn_forward_batched.shape}")

Forward pass result for EstimatorQNN: [[ 0.18131418 -0.75722239]
 [ 0.18131418 -0.75722239]].  
Shape: (2, 2)


In [31]:
sampler_qnn_forward_batched = sampler_qnn.forward(
    [sampler_qnn_input, sampler_qnn_input], sampler_qnn_weights
)

print(f"Forward pass result for SamplerQNN: {sampler_qnn_forward_batched}.  \nShape: {sampler_qnn_forward_batched.shape}")

Forward pass result for SamplerQNN: [[0.18043104 0.07078201 0.62195792 0.12682903]
 [0.18043104 0.07078201 0.62195792 0.12682903]].  
Shape: (2, 4)


Backward Pass with input gradient, setting them to True

In [32]:
estimator_qnn.input_gradients = True
sampler_qnn.input_gradients = True

In [33]:
estimator_qnn_input_grad, estimator_qnn_weight_grad = estimator_qnn.backward(
    estimator_qnn_input, estimator_qnn_weights
)

print(f"Input gradients for EstimatorQNN: {estimator_qnn_input_grad}.  \nShape: {estimator_qnn_input_grad.shape}")
print(f"Weight gradients for EstimatorQNN: {estimator_qnn_weight_grad}.  \nShape: {estimator_qnn_weight_grad.shape}")

Input gradients for EstimatorQNN: [[[ 0.14611895]
  [-0.61023657]]].  
Shape: (1, 2, 1)
Weight gradients for EstimatorQNN: [[[0.75722239]
  [0.18131418]]].  
Shape: (1, 2, 1)


In [34]:
sampler_qnn_input_grad, sampler_qnn_weight_grad = sampler_qnn.backward(
    sampler_qnn_input, sampler_qnn_weights
)

print(f"Input gradients for SamplerQNN: {sampler_qnn_input_grad}.  \nShape: {sampler_qnn_input_grad.shape}")
print(f"Weight gradients for SamplerQNN: {sampler_qnn_weight_grad}.  \nShape: {sampler_qnn_weight_grad.shape}")

Input gradients for SamplerQNN: [[[-0.06280929 -0.37676575]
  [ 0.20618381 -0.01834851]
  [-0.31113148  0.32931563]
  [ 0.16775695  0.06579863]]].  
Shape: (1, 4, 2)
Weight gradients for SamplerQNN: [[[-0.01886118 -0.35931886 -0.11301005 -0.33499331]
  [ 0.05840827  0.04849282  0.11301005 -0.09474816]
  [-0.34882586  0.37163354 -0.28085996  0.33499331]
  [ 0.30927876 -0.0608075   0.28085996  0.09474816]]].  
Shape: (1, 4, 4)
